# Analysis of rising edge jitter for AARDVARC timing performance.


Explanation or way to clean up artifacts

In [ ]:
%load_ext autoreload
%autoreload 2

## Experiment setup

Capture data using the external trigger, and make sure the trigger is synchronized with the clock.
Capturing clock-synchronized pulses requires the external trigger and the board's internal clock MUST be generated from the same source.

The synchronization removes the external sources of jitter and makes it possible to focus on the timing performance of the ASIC.
The ASIC triggers on a window-by-window (64 samples) basis, meaning an edge can arrive at any of the 64 sample positions in the window.

> **IMPORTANT!** This analysis requires a special clock file and the external clock input to be used!  
> The external trigger input and the clock driving the board MUST be derived from the same source.

## 1. Setup/Variables

In [ ]:
import logging

logger = None


def setup_logger(level=logging.INFO):
    """Setup a basic logger."""
    logger = logging.getLogger()
    handler = logging.StreamHandler()
    handler.setFormatter(
        logging.Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s")
    )
    logger.addHandler(handler)
    logger.setLevel(level)
    suppress = [
        "naludaq.UART",
        "naludaq.FTDI",
    ]
    for name in suppress:
        logging.getLogger(name).setLevel(logging.CRITICAL)
    return logger


try:
    logger.debug("logger already setup")
except Exception:
    logger = setup_logger()

### Imports

In [ ]:
from naludaq.tools.pedestals.pedestals_correcter import PedestalsCorrecter
from tqdm import tqdm

In [ ]:
from naludaq.backend import DiskAcquisition
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import scipy

## 2. Load data

### Load pedestals corrected data

Dataset analyzed is located at:   
[Dataset @ Nalu Google drive](https://drive.google.com/file/d/1szlbEnt7-hfOcPHfZuQ0G48k-HaDopgu/view?usp=drive_link)

In [ ]:
# Add the acquisition folder as a subfolder in the folder where this notebook is located.
# OR give the full path.
ACQ_PATH = r"forNaluData"

In [ ]:
# Load events into RAM
# It's possible to run this on the fly but for simplicity, the dataset is loaded into ram.
EVENT_INDEX = 1
RAMEVENTS = []
with DiskAcquisition(ACQ_PATH) as _acq:
    print("Board model:", _acq.params["model"])
    print("Number of events:", len(_acq))
    corrector = PedestalsCorrecter(_acq.params, _acq.pedestals)
    for evt in tqdm(_acq):
        corrected_event = corrector.run(evt, correct_in_place=False)
        RAMEVENTS.append(corrected_event)

### Select data channel

Pulse data captured in one channel.

In [ ]:
_first_evt = RAMEVENTS[1]

In [ ]:
# Check which channels have data in them
[i for i, x in enumerate(_first_evt["data"]) if len(x) > 0]

In [ ]:
CHANNEL = 3

## 3. Analysis

### 3.1 Plot one waveform as a sanity check

In [ ]:
plt.plot(_first_evt["data"][CHANNEL])

### Plot ALL waveforms (since population size is small)

In [ ]:
plt.figure(figsize=(12, 8))
for x in RAMEVENTS:
    plt.plot(x["data"][CHANNEL], alpha=0.3)
plt.grid()
plt.title("All overlapping events")
# plt.xlim(64,196)
plt.xlabel("sample")
plt.ylabel("ADC value")
plt.show()

## 3. Filter events

1. Filter downward going broken samples
2. Remove noisy waveform (first)

Using a negative threshold of 200

In [ ]:
def filter_events(events, channel, negative_threshold=200):
    """Filter "bad" events

    This is a custom filter removing data with visual defects found by plotting.
    It will remove events with:
    - large levels of noise
    - Negative going spikes

    Args:
        events: list like containing the events
        channel: the channel(s) to evaluate
        negative_threshold: maximum negative signal before filtering an event.

    Returns:
        output as a list of events, peaks as an equally long list of peak values.
    """
    neg_threshold = -200
    h_max = np.array([np.max(e["data"][channel]) for e in events])
    h_min = np.array([np.min(e["data"][channel]) for e in events])
    output = [events[i] for i in np.where(h_min > -200)[0]]
    peaks = h_max[np.where(h_min > neg_threshold)[0]]
    return output, peaks

In [ ]:
EVENTS, PEAKS = filter_events(RAMEVENTS, CHANNEL)

### Plot all AFTER filter

In [ ]:
plt.figure(figsize=(12, 8))
for x in EVENTS:
    plt.plot(x["data"][CHANNEL], alpha=0.3)
plt.grid()
plt.title("All overlapping FILTERED events")
plt.xticks(range(0, 320, 64))
plt.yticks(range(-50, 700, 50))
# plt.xlim(64,196)
plt.xlabel("Sample")
plt.ylabel("ADC value")
plt.show()

## Plot peak

Since the triggers are clock syncronized with the board the peaks should overlap.


### Plot peak

See if there is a pattern in the noise at the peaks.

In [ ]:
plt.figure(figsize=(12, 8))
for x in EVENTS[100:120]:
    plt.plot(x["data"][3], alpha=0.3)
plt.grid()
plt.title("20 overlapping peaks")
plt.xlim(135, 145)
plt.ylim(560, 650)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
for x in EVENTS[100:120]:
    plt.plot(x["data"][3], alpha=0.3)
plt.grid()
plt.title("20 events, 1 window before peak")
plt.xlim(64, 128)
plt.ylim(-30, 0)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
for x in EVENTS[100:120]:
    plt.plot(x["data"][3], alpha=0.3)
plt.grid()
plt.title("20 events, first window")
plt.xlim(0, 64)
plt.ylim(-30, 0)
plt.show()

## Rising edge jitter analysis

Check the jitter of the rising edge

1. calculate threshold value based on peak height
2. find crossing based on interpolation
3. Evaluate std deviation of the crossing compared to mean x-axis crossing (sample point)

In [ ]:
CHANNEL = 3

### Peak height as function of time

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(PEAKS, marker="x", linestyle="", alpha=0.4)
plt.title("Peak height as a function of time (event number)")
plt.xlabel("Event num")
plt.ylabel("ADC value @ peak (counts)")
plt.grid()
plt.show()

### Baseline as function of time

In [ ]:
def find_baseline(event):
    return np.mean(event["data"][CHANNEL][0:64])

In [ ]:
BASELINES = [find_baseline(x) for x in EVENTS]

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(BASELINES, marker="x", linestyle="", alpha=0.4)
plt.title("Baselines as a function of time (event number)")
plt.xlabel("Event num")
plt.ylabel("mean of samples 0:64 (counts)")
plt.grid()
plt.show()

### Baselines in the second window as function of time

In [ ]:
def find_baseline2(event):
    return np.mean(event["data"][CHANNEL][64:110])

In [ ]:
BASELINES2 = [find_baseline2(x) for x in EVENTS]

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(BASELINES2, marker="x", linestyle="", alpha=0.4)
plt.title("Baselines as a function of time (event number)")
plt.xlabel("Event num")
plt.ylabel("mean of samples 70:110 (counts)")
plt.grid()
plt.show()

### Peak height as function of time

In [ ]:
HEIGHTS = [x - y for x, y in zip(PEAKS, BASELINES)]

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(HEIGHTS, marker="x", linestyle="", alpha=0.4)
plt.title("Peak heights (peak - baseline) as a function of time (event number)")
plt.xlabel("Event num")
plt.ylabel("peak height (counts)")
plt.grid()
plt.show()

### Thresholds @ half max

In [ ]:
THRESHOLDS = [h * 0.5 + b for h, b in zip(HEIGHTS, BASELINES)]

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(THRESHOLDS, marker="x", linestyle="", alpha=0.4)
plt.title("THRESHOLDS (peakheight*0.5 + baseline) as a function of time (event number)")
plt.xlabel("Event num")
plt.ylabel("Threshold value (counts)")
plt.grid()
plt.show()

### Thresholds @ 30% max

In [ ]:
THRESHOLDS = [h * 0.3 + b for h, b in zip(HEIGHTS, BASELINES)]

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(THRESHOLDS, marker="x", linestyle="", alpha=0.4)
plt.title(
    "THRESHOLDS (peakheight * 0.3 + baseline) as a function of time (event number)"
)
plt.xlabel("Event num")
plt.ylabel("Threshold value (counts)")
plt.grid()
plt.show()

### Plot threshold

In [ ]:
evtnum = 300
plt.figure(figsize=(12, 8))
plt.plot(EVENTS[evtnum]["data"][CHANNEL], marker="x", linestyle="-", alpha=0.4)
plt.plot(
    [0, len(EVENTS[evtnum]["data"][CHANNEL])],
    [THRESHOLDS[evtnum], THRESHOLDS[evtnum]],
    linestyle="--",
    alpha=0.4,
)
plt.title(
    "THRESHOLDS (peakheight * 0.5 + baseline) as a function of time (event number)"
)
plt.xlabel("Event num")
plt.ylabel("ADC value (counts)")
plt.grid()
plt.show()

In [ ]:
evtnum = 300
plt.figure(figsize=(12, 8))
plt.plot(EVENTS[evtnum]["data"][CHANNEL], marker="x", linestyle="-", alpha=0.4)
plt.plot(
    [0, len(EVENTS[evtnum]["data"][CHANNEL])],
    [THRESHOLDS[evtnum], THRESHOLDS[evtnum]],
    linestyle="--",
    alpha=0.4,
)
plt.title(
    "THRESHOLDS (peakheight * 0.5 + baseline) as a function of time (event number)"
)
plt.xlabel("Event num")
plt.ylabel("ADC value (counts)")
plt.grid()
plt.xlim(130, 140)
plt.show()

### Interpolations

In [ ]:
def interpolate(event: dict, threshold: int, channel: int = 3):
    """Assumes there is an intersection

    Args:
        event: single event
        threshold: int value of threshold
        channel: single channel

    Returns:
        x-axis intersection as a float
    """
    curve = event["data"][channel]
    x2 = np.where((curve - threshold) > 0)[0][0]
    x1 = x2 - 1
    y1 = curve[x1]
    y2 = curve[x2]
    xi = x1 + ((threshold - y1) / (y2 - y1))
    return xi

### Plot histogram over intersections @ half max

In [ ]:
THRESHOLDS = [h * 0.5 + b for h, b in zip(HEIGHTS, BASELINES)]

In [ ]:
INTERPS = []
for _evt, _t in zip(EVENTS, THRESHOLDS):
    INTERPS.append(interpolate(_evt, _t))

In [ ]:
n_bins = 100
plt.figure(figsize=(12, 8))
plt.hist(INTERPS, bins=n_bins)

# Add sigma and mean lines
_mean = np.mean(INTERPS)
_std = np.std(INTERPS)
_xlabels = [round(_mean, 3)]
for i in range(1, 4):  # 1-sigma, 2-sigma, 3-sigma
    plt.axvline(
        _mean + i * _std, color="red", linestyle="--", linewidth=1, label=f"+{i}σ"
    )
    plt.axvline(
        _mean - i * _std, color="red", linestyle="--", linewidth=1, label=f"-{i}σ"
    )
    _xlabels.append(round(_mean + i * _std, 3))
    _xlabels.append(round(_mean - i * _std, 3))
plt.axvline(_mean, color="black", linestyle=":", linewidth=2, label="Mean")
plt.xticks(_xlabels)

plt.xlabel("intersect x-value")
plt.ylabel("amount")
plt.ylim(0, 1250)
# plt.grid()
plt.title("Distribution of intersections of raising edge, dynamic threshold @ half max")

### Plot histogram over intersections @ 30%

In [ ]:
THRESHOLDS = [h * 0.3 + b for h, b in zip(HEIGHTS, BASELINES)]

In [ ]:
INTERPS = []
for _evt, _t in zip(EVENTS, THRESHOLDS):
    INTERPS.append(interpolate(_evt, _t))

In [ ]:
evtnum = 300
plt.figure(figsize=(12, 8))
plt.plot(EVENTS[evtnum]["data"][CHANNEL], marker="x", linestyle="-", alpha=0.4)
plt.plot(
    [0, len(EVENTS[evtnum]["data"][CHANNEL])],
    [THRESHOLDS[evtnum], THRESHOLDS[evtnum]],
    linestyle="--",
    alpha=0.4,
)
plt.title(
    "THRESHOLDS (peakheight * 0.5 + baseline) as a function of time (event number)"
)
plt.xlabel("Event num")
plt.ylabel("mean of samples 0:64")
plt.grid()
plt.show()

In [ ]:
n_bins = 100
plt.figure(figsize=(12, 8))
plt.hist(INTERPS, bins=n_bins)

# Add sigma and mean lines
_mean = np.mean(INTERPS)
_std = np.std(INTERPS)
_xlabels = [round(_mean, 3)]
for i in range(1, 4):  # 1-sigma, 2-sigma, 3-sigma
    plt.axvline(
        _mean + i * _std, color="red", linestyle="--", linewidth=1, label=f"+{i}σ"
    )
    plt.axvline(
        _mean - i * _std, color="red", linestyle="--", linewidth=1, label=f"-{i}σ"
    )
    _xlabels.append(round(_mean + i * _std, 3))
    _xlabels.append(round(_mean - i * _std, 3))
plt.axvline(_mean, color="black", linestyle=":", linewidth=2, label="Mean")
plt.xticks(_xlabels)
plt.ylim(0, 1250)

plt.xlabel("intersect x-value")
plt.ylabel("amount")
# plt.grid()
plt.title("Distribution of intersections of raising edge, dynamic threshold @ 30%")

### Plot fixed threshold @ 300

In [ ]:
INTERPS = []
for _evt in EVENTS:
    INTERPS.append(interpolate(_evt, 300))

In [ ]:
n_bins = 100
plt.figure(figsize=(12, 8))
plt.hist(INTERPS, bins=n_bins)

# Add sigma and mean lines
_mean = np.mean(INTERPS)
_std = np.std(INTERPS)
_xlabels = [round(_mean, 3)]
for i in range(1, 4):  # 1-sigma, 2-sigma, 3-sigma
    plt.axvline(
        _mean + i * _std, color="red", linestyle="--", linewidth=1, label=f"+{i}σ"
    )
    plt.axvline(
        _mean - i * _std, color="red", linestyle="--", linewidth=1, label=f"-{i}σ"
    )
    _xlabels.append(round(_mean + i * _std, 3))
    _xlabels.append(round(_mean - i * _std, 3))
plt.axvline(_mean, color="black", linestyle=":", linewidth=2, label="Mean")
plt.xticks(_xlabels)
plt.ylim(0, 1250)

plt.xlabel("intersect x-value")
plt.ylabel("amount")
plt.title("Distribution of intersections of raising edge, fixed threshold @ 300")

### Plot fixed threshold @ 200

In [ ]:
INTERPS = []
for _evt in EVENTS:
    INTERPS.append(interpolate(_evt, 200))

In [ ]:
n_bins = 100
plt.figure(figsize=(12, 8))
plt.hist(INTERPS, bins=n_bins)

# Add sigma and mean lines
_mean = np.mean(INTERPS)
_std = np.std(INTERPS)
_xlabels = [round(_mean, 3)]
for i in range(1, 4):  # 1-sigma, 2-sigma, 3-sigma
    plt.axvline(
        _mean + i * _std, color="red", linestyle="--", linewidth=1, label=f"+{i}σ"
    )
    plt.axvline(
        _mean - i * _std, color="red", linestyle="--", linewidth=1, label=f"-{i}σ"
    )
    _xlabels.append(round(_mean + i * _std, 3))
    _xlabels.append(round(_mean - i * _std, 3))
plt.axvline(_mean, color="black", linestyle=":", linewidth=2, label="Mean")
plt.xticks(_xlabels)
plt.ylim(0, 1250)

plt.xlabel("intersect x-value")
plt.ylabel("amount")
plt.title("Distribution of intersections of raising edge, fixed threshold @ 200")

### Plot peaks, convolution

Test using convolution to cleanup noise on baseline and peak.  
Idea is to stabilize the values and get a better defined rising edge.

In [ ]:
plt.figure(figsize=(12, 8))
carrs = [
    np.array([0.1, 0.2, 0.4, 0.2, 0.1]),
    np.array([0.2, 0.2, 0.2, 0.2, 0.2]),
    np.array([0.05, 0.2, 0.5, 0.2, 0.05]),
    np.array([0.15, 0.2, 0.3, 0.2, 0.15]),
    np.array([0.15, 0.7, 0.15]),
]

en = 104
for x in EVENTS[en : en + 1]:
    in1 = x["data"][CHANNEL]
    for i, carr in enumerate(carrs):
        cevt = scipy.signal.fftconvolve(in1, carr)
        plt.plot(cevt[(len(carr) // 2) :], alpha=0.5, marker="x", label=f"kern: {i+1}")
    plt.plot(in1, color="black", linestyle="--", alpha=0.5, label="original")
plt.grid()
plt.title("Test convolut overlapping peaks")
plt.xlim(135, 145)
plt.ylim(560, 630)
plt.legend()
plt.show()

### Effect of convulution, baseline

In [ ]:
plt.figure(figsize=(12, 8))
carrs = [
    np.array([0.1, 0.2, 0.4, 0.2, 0.1]),
    np.array([0.2, 0.2, 0.2, 0.2, 0.2]),
    np.array([0.05, 0.2, 0.5, 0.2, 0.05]),
    np.array([0.15, 0.2, 0.3, 0.2, 0.15]),
    np.array([0.15, 0.7, 0.15]),
]

en = 109
for x in EVENTS[en : en + 1]:
    in1 = x["data"][CHANNEL]
    for i, carr in enumerate(carrs):
        cevt = scipy.signal.fftconvolve(in1, carr)
        plt.plot(cevt[(len(carr) // 2) :], alpha=0.5, marker="x", label=f"kern: {i+1}")
    plt.plot(in1, color="black", linestyle="--", alpha=0.5)
plt.grid()
plt.title("Test convolv, baseline")
plt.xlim(0, 64)
plt.ylim(-22, -12)
plt.legend()
plt.show()

### Effect of convolution, rising edge

In [ ]:
plt.figure(figsize=(6, 12))
carrs = [
    np.array([0.1, 0.2, 0.4, 0.2, 0.1]),
    np.array([0.2, 0.2, 0.2, 0.2, 0.2]),
    np.array([0.05, 0.2, 0.5, 0.2, 0.05]),
    np.array([0.15, 0.2, 0.3, 0.2, 0.15]),
    np.array([0.15, 0.7, 0.15]),
]

en = 109
for x in EVENTS[en : en + 1]:
    in1 = x["data"][CHANNEL]
    for i, carr in enumerate(carrs):
        cevt = scipy.signal.fftconvolve(in1, carr)
        plt.plot(cevt[(len(carr) // 2) :], alpha=0.5, marker="x", label=f"kern: {i+1}")
    plt.plot(in1, color="black", linestyle="--", alpha=0.5)
plt.grid()
plt.title("Test convolv rising edge")
plt.xlim(125, 140)
plt.ylim(0, 600)
plt.legend()
plt.show()

## Bin data based on window labels
The data seems dependent on window location, bin data based on window number at intersection.

### Redo half max but bin the data based on window where crossing happens

#### Find crossing window labels

In [ ]:
def find_baseline(event):
    return np.mean(event["data"][CHANNEL][0:64])

In [ ]:
BASELINES = [find_baseline(x) for x in EVENTS]

In [ ]:
THRESHOLDS = [h * 0.5 + b for h, b in zip(HEIGHTS, BASELINES)]

In [ ]:
from collections import defaultdict

INTERPS = defaultdict(list)
for _evt, _t in zip(EVENTS, THRESHOLDS):
    _i = interpolate(_evt, _t)
    loc_win = int(_i // 64)
    thresh_lbl = _evt["window_labels"][CHANNEL][loc_win]
    INTERPS[thresh_lbl].append(_i)

In [ ]:
plt.figure(figsize=(12, 8))
plt.bar(INTERPS.keys(), [len(x) for k, x in INTERPS.items()])
plt.title("Data amount at window locations")
plt.xlabel("windowlabel")
plt.ylabel("amount")

In [ ]:
def plot_hist(window, interps):
    n_bins = 50
    plt.figure(figsize=(12, 8))
    plt.hist(interps, bins=n_bins)

    # Add sigma and mean lines
    _mean = np.mean(interps)
    _std = np.std(interps)
    _xlabels = [round(_mean, 3)]
    for i in range(1, 4):  # 1-sigma, 2-sigma, 3-sigma
        plt.axvline(
            _mean + i * _std, color="red", linestyle="--", linewidth=1, label=f"+{i}σ"
        )
        plt.axvline(
            _mean - i * _std, color="red", linestyle="--", linewidth=1, label=f"-{i}σ"
        )
        _xlabels.append(round(_mean + i * _std, 3))
        _xlabels.append(round(_mean - i * _std, 3))
    plt.axvline(_mean, color="black", linestyle=":", linewidth=2, label="Mean")
    plt.xticks(_xlabels)
    plt.ylim(0, 150)

    plt.xlabel("intersect x-value")
    plt.ylabel("amount")
    plt.title(
        f"Distribution of intersections of raising edge, window: {window}, threshold @ 50%"
    )
    plt.show()

In [ ]:
for k, v in INTERPS.items():
    plot_hist(k, v)

### Plot mean and std dev per window location.

The specific window locations is an effect of the syncronized trigger clock and can be sued to show that the timing performance when looking at a single window is good.

In [ ]:
_means = []
_xaxis = []
_yerr = []
for k, v in sorted(INTERPS.items()):
    _means.append(np.mean(v))
    _xaxis.append(k)
    _yerr.append(np.std(v))

plt.figure(figsize=(12, 8))
plt.errorbar(
    _xaxis,
    _means,
    yerr=_yerr,
    fmt="x",
    ecolor="red",
    elinewidth=2,
    capsize=5,
    capthick=2,
)
plt.xlabel("window label")
plt.ylabel("mean intersection location (sample pt)")
plt.title("intersection mean location (std devation error bars) at window locations")
plt.yticks(np.array(range(13270, 13330, 2)) / 100)
plt.grid()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(_xaxis, _yerr, marker="x")
plt.xlabel("window number")
plt.ylabel("std deviation, [sample pt]")
plt.title("Standard deviation at window label,  binned data")
plt.grid()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data for different memory addresses

# Creating the box plot
plt.figure(figsize=(12, 8))
plt.boxplot(INTERPS.values(), labels=INTERPS.keys())
plt.xlabel("Memory Address")
plt.ylabel("Pulse Timing")
plt.title("Box Plot of Pulse Timings by Memory Address")
plt.show()